<a href="https://colab.research.google.com/github/plq-plq/-ng-d-ng-ph-n-t-n/blob/main/Chord_UDPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# chord_sim.py -- mô phỏng Chord (đơn giản, dễ đọc)
import hashlib
import random
from typing import Optional, List, Dict, Tuple

M = 8  # số bit cho vòng (0..2^M-1)
RING = 2 ** M

def hash_key(s: str) -> int:
    h = hashlib.sha1(s.encode()).digest()
    # lấy M bit thấp nhất (dùng byte slicing)
    val = int.from_bytes(h, 'big')
    return val % RING

def in_interval(x: int, a: int, b: int, inclusive_right=False) -> bool:
    # kiểm tra x ∈ (a, b) trên vòng modulo RING
    if a < b:
        return (a < x < b) or (inclusive_right and x == b)
    if a > b:
        return (x > a and x < RING) or (x >= 0 and x < b) or (inclusive_right and x == b)
    # a == b -> rỗng
    return inclusive_right and x == b

class Node:
    def __init__(self, node_id: int):
        self.id = node_id
        self.successor: 'Node' = self
        self.predecessor: Optional['Node'] = None
        self.finger: List['Node'] = [self for _ in range(M)]
        self.data: Dict[int, str] = {}  # key->value

    def __repr__(self):
        return f"Node({self.id})"

    # --- core operations ---
    def find_successor(self, id_: int, hop_counter: Optional[List[int]] = None) -> 'Node':
        if hop_counter is not None:
            hop_counter[0] += 1
        if self == self.successor:
            return self
        if in_interval(id_, self.id, self.successor.id, inclusive_right=True):
            return self.successor
        n0 = self.closest_preceding_node(id_)
        if n0 == self:
            # fallback
            return self.successor
        return n0.find_successor(id_, hop_counter)

    def closest_preceding_node(self, id_: int) -> 'Node':
        for i in reversed(range(M)):
            finger = self.finger[i]
            if finger is None:
                continue
            if in_interval(finger.id, self.id, id_):
                return finger
        return self

    def join(self, existing: Optional['Node']):
        if existing is None:
            # tạo mạng mới
            self.predecessor = None
            self.successor = self
            for i in range(M):
                self.finger[i] = self
        else:
            self.predecessor = None
            self.successor = existing.find_successor(self.id)
            # không chuyển data ở đây; caller sẽ rebalance

    def stabilize(self):
        x = self.successor.predecessor
        if x and in_interval(x.id, self.id, self.successor.id):
            self.successor = x
        self.successor.notify(self)

    def notify(self, n: 'Node'):
        if (self.predecessor is None) or in_interval(n.id, self.predecessor.id, self.id):
            self.predecessor = n

    def fix_fingers(self):
        for i in range(M):
            target = (self.id + 2**i) % RING
            self.finger[i] = self.find_successor(target)

    # helper để insert key
    def put(self, key: str, value: str):
        k = hash_key(key)
        dest = self.find_successor(k)
        dest.data[k] = value
        return dest.id

    def get(self, key: str) -> Tuple[Optional[str], int]:
        k = hash_key(key)
        hops = [0]
        dest = self.find_successor(k, hops)
        val = dest.data.get(k)
        return val, hops[0]

# --- tiện ích cho mô phỏng ---

def create_ring(node_ids: List[int]) -> List[Node]:
    nodes = [Node(i) for i in sorted(node_ids)]
    if not nodes:
        return []
    # khởi tạo first node
    first = nodes[0]
    first.join(None)
    # thêm các node còn lại
    for n in nodes[1:]:
        n.join(first)
        # gọi stabilize/fix_fingers toàn mạng một vài lần
        for _ in range(3):
            for nn in nodes:
                nn.stabilize()
                nn.fix_fingers()
    # nhỏ gọn: trả về list nodes
    return nodes

# hàm rebalance dữ liệu: gán lại keys cho successor tương ứng (dùng sau khi join/leave)
def rebalance(nodes: List[Node], all_keys: Dict[int,str]):
    # xóa data cũ
    for n in nodes:
        n.data.clear()
    for k, v in all_keys.items():
        # mỗi key lưu trên successor của k
        target = nodes[0].find_successor(k)
        for n in nodes:
            if n.id == target.id:
                n.data[k] = v
                break

# tính số hops trung bình cho một tập truy vấn
def measure_hops(nodes: List[Node], queries: List[str]) -> Tuple[float, int, int]:
    total_hops = 0
    misses = 0
    for q in queries:
        val, hops = nodes[0].get(q)
        total_hops += hops
        if val is None:
            misses += 1
    avg_hops = total_hops / len(queries)
    return avg_hops, total_hops, misses

# Ví dụ sử dụng trong test
if __name__ == '__main__':
    # tạo mạng ban đầu
    node_ids = [10, 50, 120]
    nodes = create_ring(node_ids)
    # tạo một số keys
    keys = {hash_key(f'k{i}'): f'v{i}' for i in range(30)}
    rebalance(nodes, keys)

    # đo hops trước khi thêm node
    queries = [f'k{random.randint(0,29)}' for _ in range(100)]
    avg_hops_before, _, misses_before = measure_hops(nodes, queries)
    print('Avg hops before:', avg_hops_before, 'misses:', misses_before)

    # thêm node
    new_node = Node(200)
    nodes.append(new_node)
    new_node.join(nodes[0])
    for _ in range(5):
        for n in nodes:
            n.stabilize(); n.fix_fingers()
    rebalance(nodes, keys)
    avg_hops_after, _, misses_after = measure_hops(nodes, queries)
    print('Avg hops after:', avg_hops_after, 'misses:', misses_after)


Avg hops before: 1.0 misses: 0
Avg hops after: 1.0 misses: 0
